# User features

## Import packages

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import random
random.seed(32)
import gensim
from gensim import corpora, models
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

## Import data

In [2]:
# authors = pd.read_csv('/home/sophia/ma_py/author_profiles.csv')

df = pd.read_pickle("preprocessed_author.pkl")
# df = pre_df.set_index(['index', 'author'])
# df.head(10)
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 422 entries, 0 to 421
Data columns (total 26 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   index                  422 non-null    int64  
 1   author                 422 non-null    object 
 2   complete_body          422 non-null    object 
 3   doc_body               422 non-null    object 
 4   probody                422 non-null    object 
 5   tokens                 422 non-null    object 
 6   senttokens             422 non-null    object 
 7   agreeableness          422 non-null    float64
 8   openness               422 non-null    float64
 9   conscientiousness      422 non-null    float64
 10  extraversion           422 non-null    float64
 11  neuroticism            422 non-null    float64
 12  agree                  422 non-null    int64  
 13  openn                  422 non-null    int64  
 14  consc                  422 non-null    int64  
 15  extra 

In [3]:
df.tail()

,index,author,complete_body,doc_body,probody,tokens,senttokens,agreeableness,openness,conscientiousness,...,neuro,all_utc,mean_controversiality,mean_gilded,num_subreddits,subreddit_dist,weekday_dist,month_dist,year_dist,all_lang
417,424,xanthraxoid,I'd really like this video to include some inf...,[I'd really like this video to include some in...,[would really like video include information c...,"[[would, really, like, video, include, informa...",[[I'd really like this video to include some i...,86.0,45.0,8.0,...,1,"[1469892161, 1486826547, 1498046590, 1550346594]",0.0,0.0,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0]","[0, 2, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0]","[0, 1, 2, 0, 1]",0 0 3 0
418,425,xenomouse,"You're a guy, aren't you? I can definitely see...","[You're a guy, aren't you? I can definitely se...",[guy not definitely see would make boy scene f...,"[[guy, not, definitely, see, would, make, boy,...","[[You're a guy, aren't you?, I can definitely ...",26.0,93.0,49.0,...,0,"[1506710219, 1502740906, 1517847908, 1506874589]",0.0,0.0,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0]","[0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0]","[0, 0, 3, 1, 0]",0 0 0 0
419,426,xeroctr3,man even the thought of it makes me depressed....,[man even the thought of it makes me depressed...,[man even thought makes depressed loving someo...,"[[man, even, thought, makes, depressed, loving...",[[man even the thought of it makes me depresse...,3.0,75.0,27.0,...,1,[1521414051],0.0,0.0,1,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 1, 0]",0
420,427,xzack18,Not all of us are out to kill,[Not all of us are out to kill],[not us kill],"[[not, us, kill]]",[[Not all of us are out to kill]],4.0,19.0,11.0,...,0,[1533749569],0.0,0.0,1,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0]","[0, 0, 0, 1, 0]",0
421,428,zugzwang_03,Institutions should accommodate religious or s...,[Institutions should accommodate religious or ...,[institutions accommodate religious serious me...,"[[institutions, accommodate, religious, seriou...",[[Institutions should accommodate religious or...,10.0,41.0,86.0,...,0,"[1514216199, 1459000262, 1500701643, 151759545...",0.0,0.0,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0]","[0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 1]","[0, 3, 2, 1, 0]",0 0 0 0 0 0


## LDA

In [11]:
# https://rstudio-pubs-static.s3.amazonaws.com/79360_850b2a69980c4488b1db95987a24867a.html

# problem: probody does not have punctuation anymore
# multicore instead of lda model seems to be slower????

def apply_lda(df, number, name):
    print("Preprocessing for LDA...")
    lst = []
    # filter out stopwords including negations for topic modeling
#     stopwordList = stopwords.words('english')
#     df['ldatokens'] = df['probody'].apply(lambda x: ' '.join([x for x in x.split() if (x not in stopwordList)]))
#     df['ldatokens'] = df['ldatokens'].apply(lambda x: (word_tokenize(x)))
#     print(df.iloc[421]['ldatokens'])
    print("Start LDA...")
    for row in tqdm(df['tokens']):
        if len(row) < 2:
            lst.append([0])
        else:
            dictionary = corpora.Dictionary(row)
            corpus = [dictionary.doc2bow(text) for text in row]
            ldamodel = gensim.models.LdaMulticore(corpus, num_topics=number, id2word = dictionary, passes=20, workers=15)
            result = ldamodel.print_topics(num_topics=number, num_words=1)
            lst.append(result)
    df[name] = lst
    return df

# print(df.iloc[220]['tokens'])
# print(len(df.iloc[220]['tokens']))
df = apply_lda(df, 3, "ldathree")
df['ldathree']

Preprocessing for LDA...
Start LDA...


  0%|          | 0/422 [00:00<?, ?it/s]

0                                                    [0]
1                                                    [0]
2      [(0, 0.051*","), (1, 0.035*"racism"), (2, 0.03...
3      [(0, 0.210*"sorry"), (1, 0.143*"nothing"), (2,...
4      [(0, 0.071*"day"), (1, 0.115*"get"), (2, 0.125...
                             ...                        
417    [(0, 0.057*"not"), (1, 0.039*"air"), (2, 0.012...
418    [(0, 0.041*"would"), (1, 0.047*"not"), (2, 0.0...
419                                                  [0]
420                                                  [0]
421    [(0, 0.025*"someone"), (1, 0.021*"first"), (2,...
Name: ldathree, Length: 422, dtype: object

# Wrapper

In [12]:
def extract_userfeatures(df):
    df = apply_lda(df, 50, "ldafifty")
    df = apply_lda(df, 100, "ldahundred")
    return df

In [13]:
user_feat_df = extract_userfeatures(df)

Preprocessing for LDA...
Start LDA...


  0%|          | 0/422 [00:00<?, ?it/s]

Preprocessing for LDA...
Start LDA...


  0%|          | 0/422 [00:00<?, ?it/s]

In [ ]:
user_feat_df.to_pickle("user_feat_df.pkl")